In [1]:
import numpy as np
from numpy import genfromtxt

elec_data = genfromtxt('data/elec_final_data_hackathon.csv', delimiter=',', dtype = str)
water_data = genfromtxt('data/water_final_data_hackathon.csv', delimiter=',', dtype = str)
weather_data = genfromtxt('data/2017-2020_weather_data.csv', delimiter=',', dtype = str)

In [2]:
mueller_weather_data = []
for row in weather_data:
	if row[1] == "30.292432" and row[2] == "-97.699662":
		mueller_weather_data.append(row)
mueller_weather_data = np.array(mueller_weather_data)

In [3]:
print(len(mueller_weather_data))

35427


In [4]:
matches_dict = dict()

skip_head = True
for row in elec_data:
    if skip_head:
        skip_head = False
        continue
    l1 = row[0].split(" ")
    house_id = row[1]
    year = l1[0].split("-")[0]
    month = l1[0].split("-")[1]
    date = l1[0].split("-")[2]
    hour = l1[1].split(":")[0]
    timestamp = (year, month, date, hour, house_id)
    val = []
    val.append(row[2])
    val.append(row[3])
    matches_dict[timestamp] = val

skip_head = True
for row in water_data:
    if skip_head:
        skip_head = False
        continue
    l1 = row[0].split(" ")
    house_id = row[1]
    year = l1[0].split("-")[0]
    month = l1[0].split("-")[1]
    date = l1[0].split("-")[2]
    hour = l1[1].split(":")[0]
    timestamp = (year, month, date, hour, house_id)
    if timestamp in matches_dict:
        matches_dict[timestamp].append(row[2])

sum = 0
for key in matches_dict:
    if len(matches_dict[key]) == 3:
        sum += 1
print(sum)

    

924397


In [32]:
print(mueller_weather_data[-1])

['"2020-12-31 23:00:00"' '30.292432' '-97.699662' 'NULL' '"Overcast"'
 'NULL' 'NULL' '37.37' 'NULL' '34.34' 'NULL' '0.89' 'NULL' '10' 'NULL'
 '30.42' 'NULL' '1009.3' 'NULL' '9.91' 'NULL' '0.92' 'NULL' '258' '0'
 'NULL' '0' 'NULL' 'NULL' 'NULL']


In [38]:
sample_size = len(mueller_weather_data)
time_parsed_data_weather = np.zeros([sample_size,17])
time_parsed_data_weather = time_parsed_data_weather.astype('str')
for i in range(0,sample_size):
    time1 = mueller_weather_data[i][0]
    [ymd,hmm] = time1.split()
    [y,m,d] = ymd.split('-')
    y = y[1:] #hard coding out the 
    [h,min,sec] = hmm.split(":")
    all_might_bool_array = np.array([False,True,True,False,True,False,False,False,False,True,False,True,False,True,False,True,False,True,False,True,False,True,False,True,True,False,True,False,False,False])
    temp = mueller_weather_data[i][all_might_bool_array]
    temp2 = np.concatenate(([y,m,d,h],temp))
    time_parsed_data_weather[i,:] = temp2

In [39]:
print(time_parsed_data_weather[-1])

['2020' '12' '31' '23' '30.292432' '-97.699662' '"Overcast"' '34.34'
 '0.89' '10' '30.42' '1009.3' '9.91' '0.92' '258' '0' '0']


In [42]:
elec_wtr_dict = dict()
for key in matches_dict:
    if len(matches_dict[key]) == 3:
        elec_wtr_dict[key] = matches_dict[key]

In [43]:
matches_dict = None

In [44]:
timestamp_dict = dict()
for key in elec_wtr_dict:
    temp = (key[0], key[1], key[2], key[3])
    if temp not in timestamp_dict:
        secondary_dict = dict()
        secondary_dict[key[4]] = elec_wtr_dict[key]
        timestamp_dict[temp] = secondary_dict
    else:
        timestamp_dict[temp][key[4]] = elec_wtr_dict[key]



In [45]:
print(time_parsed_data_weather[-1][4:])

['30.292432' '-97.699662' '"Overcast"' '34.34' '0.89' '10' '30.42'
 '1009.3' '9.91' '0.92' '258' '0' '0']


In [46]:
count = 0
for row in time_parsed_data_weather:
    ts = (row[0], row[1], row[2], row[3])
    if ts in timestamp_dict:
        count += 1
        timestamp_dict[ts]["weather"] = row[4:]
print(count)

33869


In [49]:
print(type(time_parsed_data_weather[-1][4:]))

<class 'numpy.ndarray'>


In [55]:
preprocessed_list = []
for timestamp in timestamp_dict:
    ts_list = list(timestamp)
    if "weather" in timestamp_dict[timestamp]:
        weather_data = timestamp_dict[timestamp]["weather"]
        for second_key in timestamp_dict[timestamp]:
            if second_key != "weather":
                ts_list.append(second_key)
                new_ts_list = ts_list + timestamp_dict[timestamp][second_key]
                ts_list.remove(second_key)
                preprocessed_list.append(np.concatenate((np.array(new_ts_list), weather_data), axis=None))
            
print(len(preprocessed_list))
    

922747


In [58]:
print(preprocessed_list[0])
print(preprocessed_list[-1])
print(type(preprocessed_list[0][0]))

['2017' '09' '04' '15' '59.0' '2.555' '0.0' '72.832' '30.292432'
 '-97.699662' '"Clear"' '65.18' '0.44' '9.92' '92.09' '1015.44' '4.99'
 '0.02' '152' '0' '0']
['2020' '10' '03' '22' '7989.0' '1.1340000000000001' '-0.008' '0.0'
 '30.292432' '-97.699662' '"Clear"' '60.31' '0.73' '10' '69.22' '1017.7'
 '5.03' '0' '145' '0' '0']
<class 'numpy.str_'>


In [59]:
import csv

with open('data/preprocessed_data.csv', 'w') as f:
    csvwriter = csv.writer(f)
    csvwriter.writerows(preprocessed_list)